I dropped transactions where CustomerID was missing, since those couldn’t be linked to customer behavior.
I also removed rows with missing descriptions or zero/negative prices, except for return cases.

Many customers returned some items after purchasing multiple products.
I matched purchases and returns based on CustomerID, InvoiceDate, and StockCode.
Instead of removing all such entries, I calculated the net quantity for each product — for example, if someone bought 5 items and returned 3, I kept the remaining 2.
If the return canceled out the full purchase (net quantity zero or negative), I removed it.

Some entries like StockCode 'POST', 'M', or 'D' were not actual products — I kept them only when the description indicated they were real products (like 'Pencil'), and dropped them if they represented metadata or manual entries (like 'Manual', 'Adjustment').

Converted InvoiceDate into separate fields like invoice_month and invoice_day_of_week to help analyze seasonal and weekly patterns.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
print(f'this data have {df.shape[0]} entries and {df.shape[1]} columns')

this data have 541909 entries and 8 columns


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


Every Columns has the Correct data types but there is some missing values.

In [5]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Description and CustomerID Contains NAN Values.

In [6]:
data = df.dropna()
data.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Initially there is some NAN Values in customerId and description and i dropped because NAN contains in Customer id thats why

In [7]:
data.duplicated().sum()

5225

In [8]:
data[data.duplicated()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,2010-12-01 11:45:00,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,2010-12-01 11:49:00,2.95,17920.0,United Kingdom
...,...,...,...,...,...,...,...,...
541675,581538,22068,BLACK PIRATE TREASURE CHEST,1,2011-12-09 11:34:00,0.39,14446.0,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,2011-12-09 11:34:00,2.49,14446.0,United Kingdom
541692,581538,22992,REVOLVER WOODEN RULER,1,2011-12-09 11:34:00,1.95,14446.0,United Kingdom
541699,581538,22694,WICKER STAR,1,2011-12-09 11:34:00,2.10,14446.0,United Kingdom


I found some duplicates but that is InvoiceNo so no problem i leave that.

In [9]:
data['invoice_mnth'] = data['InvoiceDate'].dt.month
data['invoicedayweek'] = data['InvoiceDate'].dt.day_name()

In [10]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoice_mnth,invoicedayweek
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,12,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,12,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12,Wednesday


I created new columns invoice_month and inviceweek for further analysis.

In [11]:
num = data.select_dtypes(include = ['float64','int64'])
num.drop(columns = 'CustomerID',axis = 1, inplace = True)
chk_col = num.columns.tolist()
print(chk_col)

['Quantity', 'UnitPrice']


In [12]:

for col in chk_col:
    has = (data[col]<0).any()
    if has:
        for i in data.index:
            if data.loc[i,col] < 0:
                data.loc[i,col] = data.loc[i,col]*-1

In [13]:
for col in chk_col:
    has = (data[col]<0).any()
    if has:
        print(f'there is some neg value in {col}')
    else:
        print(f'there is no some neg value in {col}')

there is no some neg value in Quantity
there is no some neg value in UnitPrice


In [14]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'invoice_mnth', 'invoicedayweek'],
      dtype='object')

In [15]:
for col in data.columns:
  s = data[col].nunique()
  print(f'number of unique values {s} in {col}')
  print("-"*30)

number of unique values 22190 in InvoiceNo
------------------------------
number of unique values 3684 in StockCode
------------------------------
number of unique values 3896 in Description
------------------------------
number of unique values 310 in Quantity
------------------------------
number of unique values 20460 in InvoiceDate
------------------------------
number of unique values 620 in UnitPrice
------------------------------
number of unique values 4372 in CustomerID
------------------------------
number of unique values 37 in Country
------------------------------
number of unique values 12 in invoice_mnth
------------------------------
number of unique values 6 in invoicedayweek
------------------------------


In [16]:
data['Total_revenue'] = data['Quantity']*data['UnitPrice']

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   InvoiceNo       406829 non-null  object        
 1   StockCode       406829 non-null  object        
 2   Description     406829 non-null  object        
 3   Quantity        406829 non-null  int64         
 4   InvoiceDate     406829 non-null  datetime64[ns]
 5   UnitPrice       406829 non-null  float64       
 6   CustomerID      406829 non-null  float64       
 7   Country         406829 non-null  object        
 8   invoice_mnth    406829 non-null  int32         
 9   invoicedayweek  406829 non-null  object        
 10  Total_revenue   406829 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int32(1), int64(1), object(5)
memory usage: 51.8+ MB


In [18]:
(data['Total_revenue'] < 0).any()

False

In [19]:
# Grouping the Quantity Column.

for y in data.index:
  if data.loc[y, 'Quantity'] <= 10:
    data.loc[y, 'grp_qnty'] =  '1-10'
  elif data.loc[y, 'Quantity'] <= 50:
    data.loc[y, 'grp_qnty'] =  '11-50'
  elif data.loc[y, 'Quantity'] <= 100:
    data.loc[y, 'grp_qnty'] =  '51-100'
  elif data.loc[y, 'Quantity'] <= 200:
    data.loc[y, 'grp_qnty'] =  '101-200'
  elif data.loc[y, 'Quantity'] <= 500:
    data.loc[y, 'grp_qnty'] =  '201-500'
  elif data.loc[y, 'Quantity'] <= 1000:
    data.loc[y, 'grp_qnty'] =  '501-1000'
  else:
    data.loc[y, 'grp_qnty'] =  '1000+'

In [20]:
for t in data.index:
  if data.loc[t, 'Total_revenue'] <= 1000:
    data.loc[t, 'grp_rev'] =  '1-1000'
  elif data.loc[t, 'Total_revenue'] <= 5000:
    data.loc[t, 'grp_rev'] =  '1001-5000'
  elif data.loc[t, 'Total_revenue'] <= 10000:
    data.loc[t, 'grp_rev'] =  '5001-10000'
  elif data.loc[t, 'Total_revenue'] <= 20000:
    data.loc[t, 'grp_rev'] =  '10001-20000'
  elif data.loc[t, 'Total_revenue'] <= 50000:
    data.loc[t, 'grp_rev'] =  '20001-50000'
  elif data.loc[t, 'Total_revenue'] <= 100000:
    data.loc[t, 'grp_rev'] =  '50001-100000'
  else:
    data.loc[t, 'grp_rev'] =  '100000+'

above columns are useful for further analysis

The above statistics are not realistic may we shrinked lot of data this is not a right option we have try other ways

Before that we have to remove some of the stock codes like post,M etc.

In [21]:
specific_stockcode = data[data['StockCode'].str.match(r'[A-Za-z]+', na = False)]
unique_code = specific_stockcode['StockCode'].unique().tolist()
unique_code

['POST', 'D', 'C2', 'M', 'BANK CHARGES', 'PADS', 'DOT', 'CRUK']

In [22]:
cldata = data.copy()

In [23]:
for code in unique_code:
    filt = data[data['StockCode'] == code]
    sc = filt['StockCode'].unique()[0]
    dc = filt['Description'].unique()[0]
    cldata = cldata[~((cldata['StockCode'] == sc) & (cldata['Description'] == dc))]

In [24]:
cldata[cldata['StockCode'].str.match(r'[A-Za-z]+', na = False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,invoice_mnth,invoicedayweek,Total_revenue,grp_qnty,grp_rev


In [25]:
cldata['InvoiceNo_Alpha'] = cldata['InvoiceNo'].str.extract(r'^([A-Za-z]+)', expand=False)

In [26]:
cldata[cldata['InvoiceNo'].str.match(r'^([A-Za-z]+)', na = False)]['InvoiceNo_Alpha'].unique()

array(['C'], dtype=object)

In [ ]:
for i in cldata.index:
    if cldata.loc[i, 'InvoiceNo_Alpha'] == 'C':
        cldata.loc[i, 'InvoiceIdentity'] = 'Cancellation/Return'
    else:
        cldata.loc[i, 'InvoiceIdentity'] = 'Normal_Product'

In [ ]:
cldata['InvoiceIdentity'].unique()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
purchase = cldata[cldata['InvoiceIdentity'] == 'Normal_Product']
cancelled = cldata[~(cldata['InvoiceIdentity'] == 'Normal_Product')]

In [ ]:
match_returns = pd.merge(purchase, cancelled, on = ['CustomerID', 'StockCode'], suffixes = ('_p', '_c'))

In [ ]:
cldata.loc[cldata['InvoiceIdentity'] != 'Normal_Product', 'Quantity'] = -cldata.loc[cldata['InvoiceIdentity'] != 'Normal_Product', 'Quantity'].abs()

In [ ]:
returns = cldata[cldata['InvoiceIdentity'] != 'Normal_Product']
purchase = cldata[cldata['InvoiceIdentity'] == 'Normal_Product']

In [ ]:
returns['Quantity'] = returns['Quantity'].abs()

In [ ]:
merge = pd.merge(returns, purchase, on = ['StockCode', 'CustomerID'], suffixes=('_return', '_purchase'))

In [ ]:
valid_return = merge[merge['InvoiceDate_return'] > merge['InvoiceDate_purchase']]

In [ ]:
merge['day_diff'] = (merge['InvoiceDate_return'] - merge['InvoiceDate_purchase']).dt.days

In [ ]:
merge_filt = merge[(merge['day_diff'] >= 0) & (merge['day_diff'] <= 20) ]

In [ ]:
merge_filt['Adj_qnt'] =(merge_filt['Quantity_purchase'] - merge_filt['Quantity_return'])

In [ ]:
adjusted = merge_filt[merge_filt['Adj_qnt'] > 0]
full_ret =  merge_filt[merge_filt['Adj_qnt'] <= 0]

In [ ]:
full_ret.head()

In [ ]:
adjusted['Quantity_purchase'] = adjusted['Adj_qnt']

In [ ]:
to_remove = full_ret[['StockCode', 'CustomerID', 'InvoiceDate_purchase']]

In [ ]:
cleaned = purchase.merge(to_remove, left_on = ['StockCode', 'CustomerID', 'InvoiceDate'], right_on = ['StockCode', 'CustomerID', 'InvoiceDate_purchase'], how = 'left', indicator = True).query('_merge =="left_only"').drop(columns=['InvoiceDate_purchase', '_merge'])

In [ ]:
adjusted.columns

In [ ]:
to_remove2 = adjusted[['StockCode','CustomerID','InvoiceDate_purchase']].rename(columns = {'InvoiceDate_purchase':'InvoiceDate'})

In [ ]:
cleaned1 = pd.merge(cleaned, to_remove2, on = ['StockCode', 'CustomerID', 'InvoiceDate'], how = 'left', indicator = True).query('_merge == "left_only"').drop(columns=['_merge'])

In [ ]:
cleaned1.head()

In [ ]:
cleaned1.columns

In [ ]:
adjusted.columns

In [ ]:
adjusted.rename(columns = {'InvoiceNo_purchase' : 'InvoiceNo', 'Description_purchase' : 'Description', 'Quantity_purchase' : 'qnt',
       'InvoiceDate_purchase' : 'InvoiceDate', 'UnitPrice_purchase' : 'UnitPrice', 'Country_purchase' : 'Country',
       'invoice_mnth_purchase' : 'invoice_mnth', 'invoicedayweek_purchase' : 'invoicedayweek',
       'Total_revenue_purchase' : 'Total_revenue', 'grp_qnty_purchase' : 'grp_qnty', 'grp_rev_purchase' : 'grp_rev',
       'InvoiceNo_Alpha_purchase' : 'InvoiceNo_Alpha', 'InvoiceIdentity_purchase' : 'InvoiceIdentity', 
       'Adj_qnt' : 'Quantity'}, inplace = True)

In [ ]:
cleaned1.columns

In [ ]:
ad_filt = adjusted[['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice','CustomerID',
       'Country', 'invoice_mnth', 'invoicedayweek', 'Total_revenue',
       'grp_qnty', 'grp_rev', 'InvoiceNo_Alpha', 'InvoiceIdentity',]]

In [ ]:
final_data = pd.concat([cleaned1, ad_filt], ignore_index = True)

In [ ]:
final_data.isnull().sum()

In [ ]:
final_data.shape

In [ ]:
final_data.describe()

In [ ]:
final_data.to_csv('cleaned_data', index = False)

In [ ]:
cldata[cldata['Quantity'] < 0].shape

In [ ]:
return_data = cldata[cldata['InvoiceIdentity'] == 'Cancellation/Return']

In [ ]:
return_data.to_csv('return_data', index = False)

In [ ]:
data['StockCode']